# Conformer Minimization

This example will show how to optimize a conformer of paracetamol.

Load in a paracetamol molecule, generate a conformer for it, and perturb the conformer to ensure it needs minimization.

In [1]:
import openff.toolkit
import openff.units
import torch

molecule = openff.toolkit.Molecule.from_smiles("CC(=O)NC1=CC=C(C=C1)O")
molecule.generate_conformers(n_conformers=1)

conformer = torch.tensor(molecule.conformers[0].m_as(openff.units.unit.angstrom)) * 1.10
conformer.requires_grad = True

We specify that the gradient of the conformer is required so that we can optimize it using PyTorch.

Parameterize the molecule using OpenFF Interchange and convert it into a PyTorch tensor representation.

In [2]:
import openff.interchange

interchange = openff.interchange.Interchange.from_smirnoff(
    openff.toolkit.ForceField("openff-2.1.0.offxml"),
    molecule.to_topology(),
)

import smirnoffee.ff

force_field, [topology] = smirnoffee.ff.convert_interchange(interchange)

We can minimize the conformer using any of PyTorch's optimizers. 

In [3]:
import smirnoffee.potentials

optimizer = torch.optim.Adam([conformer], lr=0.02)

for epoch in range(75):
    energy = smirnoffee.potentials.compute_energy(
        topology.parameters, conformer, force_field
    )
    energy.backward()

    optimizer.step()
    optimizer.zero_grad()

    if epoch % 5 == 0 or epoch == 74:
        print(f"Epoch {epoch}: E={energy.item()} kcal / mol")

2023-09-16 07:02:45.103 python[21951:179166] apply_selection_policy_once: avoid use of removable GPUs (via (null):GPUSelectionPolicy->avoidRemovable)


Epoch 0: E=102.10975646972656 kcal / mol
Epoch 5: E=7.088310718536377 kcal / mol
Epoch 10: E=-18.331079483032227 kcal / mol
Epoch 15: E=-22.182260513305664 kcal / mol
Epoch 20: E=-30.369123458862305 kcal / mol
Epoch 25: E=-36.8104133605957 kcal / mol
Epoch 30: E=-38.51783752441406 kcal / mol
Epoch 35: E=-40.50504684448242 kcal / mol
Epoch 40: E=-42.084754943847656 kcal / mol
Epoch 45: E=-42.191986083984375 kcal / mol
Epoch 50: E=-42.378273010253906 kcal / mol
Epoch 55: E=-42.67677688598633 kcal / mol
Epoch 60: E=-42.79990005493164 kcal / mol
Epoch 65: E=-42.942508697509766 kcal / mol
Epoch 70: E=-43.037193298339844 kcal / mol
Epoch 74: E=-43.084136962890625 kcal / mol


We can then re-store the optimized conformer back into the molecule. Here we add the conformer to the molecule's conformer list, but we could also replace the original conformer.

In [4]:
molecule.add_conformer(conformer.detach().numpy() * openff.units.unit.angstrom)
molecule.visualize(backend="nglview")

NGLWidget(max_frame=1)